# Passing Networks
Visualization and calculation the centralized nature of a game for teams involved.  Normally this is evaluated for one of the teams in a given game.  This helps to show the average shape of the team throughout the game.  Normally stopped at the first substution.

## Imports
This version will deviate from the Soccermatics approach due to the mplsoccer library throwing an error when making a call to Statsbomb.  Instead, the statsbomb data will be called directly for the dataframe from the statsbomb library.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mplsoccer import Pitch, Sbopen, VerticalPitch
from statsbombpy import sb

## Opening and creating the dataset
Using the sb parser in place of SBopen in mplsoccer the event data is pulled out. Calling on the event level data and inputting the id of the game as a parameter loads the game data. The event data is stored in a dataframe df. From this dataframe, the next step will be to filter for passes not including throw ins are left in the frame.

In [2]:
# get event data
df = sb.events(match_id=69301)

In [7]:
df.query('type=="Pass"').loc[:,lambda df: df.columns.str.startswith('pass')].iloc[0]

pass_aerial_won                      NaN
pass_angle                      0.442374
pass_assisted_shot_id                NaN
pass_body_part                Right Foot
pass_cross                           NaN
pass_cut_back                        NaN
pass_deflected                       NaN
pass_end_location           [61.9, 40.9]
pass_goal_assist                     NaN
pass_height                  Ground Pass
pass_inswinging                      NaN
pass_length                      2.10238
pass_no_touch                        NaN
pass_outcome                         NaN
pass_recipient           Francesca Kirby
pass_shot_assist                     NaN
pass_straight                        NaN
pass_switch                          NaN
pass_technique                       NaN
pass_through_ball                    NaN
pass_type                       Kick Off
Name: 6, dtype: object

## Preparing the data
For passing networks only use accurate/successful passes made by a team until the first substitution. This is the most basic use case, and there are several possible variations using passing networks. They require information about pass start and end location as well as player who made and received the pass. To make the vizualisation clearer annotate the players using their surname. 

In [4]:
# find index of first sub for England
sub = df.loc[df["type"] == "Substitution"].loc[df["team"] == "England Women's"].iloc[0]["index"]
sub

1764

In [25]:
#make df with successfull passes by England until the first substitution
# mask_england = (df.type_name == 'Pass') & (df.team_name == "England Women's") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")


(df
 .query('type=="Pass" and team.str.startswith("Eng") and pass_outcome.isnull() and pass_type != "Throw-in"')
 .loc[lambda df: df.index<sub]
 .assign(x = lambda df: df.location.apply(pd.Series)[0],
         y = lambda df: df.location.apply(pd.Series)[1],
         xf = lambda df: df.pass_end_location.apply(pd.Series)[0],
         yf = lambda df: df.pass_end_location.apply(pd.Series)[1])
 .loc[:,['x', 'y', 'xf', 'yf', 'player', 'pass_recipient']]
 .assign(player_name = lambda df: df.player.str.split().str[1],
         player_recipient_name = lambda df: df.pass_recipient.str.split().str[1])
)

,x,y,xf,yf,player,pass_recipient,player_name,player_recipient_name
6,60.0,40.0,61.9,40.9,Ellen White,Francesca Kirby,White,Kirby
7,82.6,37.6,78.4,58.5,Francesca Kirby,Lucy Bronze,Kirby,Bronze
8,84.7,65.3,98.9,77.9,Lucy Bronze,Nikita Parris,Bronze,Parris
9,94.7,75.5,93.3,64.3,Nikita Parris,Jill Scott,Parris,Scott
21,5.2,8.3,10.8,5.5,Jade Moore,Bethany Mead,Moore,Mead
...,...,...,...,...,...,...,...,...
920,49.3,68.5,54.0,42.0,Stephanie Houghton,Jade Moore,Houghton,Moore
921,60.6,37.5,70.2,25.2,Jade Moore,Francesca Kirby,Moore,Kirby
922,77.0,20.1,90.5,23.9,Francesca Kirby,Karen Julia Carney,Kirby,Julia
923,91.8,19.4,102.1,5.4,Karen Julia Carney,Alex Greenwood,Julia,Greenwood
